# Classifying multi-label texts with Logistic Regression
Izzy
Started on april 10

-- inspired by:

Sergei Fironov

In [ ]:
# importing libraries
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
import re
# Fork of Sergei Fironov's script CNN GLOVE300 3-OOF 3 epochs
import os
os.environ['OMP_NUM_THREADS'] = '4'
import tensorflow as tf
from keras.models import Model
from keras.layers import Dense, Embedding, Input, Concatenate, Conv1D, Activation, TimeDistributed, Flatten, RepeatVector, Permute,multiply
from keras.layers import LSTM, Bidirectional, GlobalMaxPool1D, Dropout, GRU, GlobalAveragePooling1D, MaxPooling1D, SpatialDropout1D, BatchNormalization
from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.model_selection import KFold
from sklearn import metrics
# def RNN():
#     inputs = Input(name='inputs',shape=[max_len])
#     layer = Embedding(max_words,100,input_length=max_len)(inputs)
#     layer = LSTM(64)(layer)
#     layer = Dense(256,name='FC1')(layer)
#     layer = Activation('relu')(layer)
#     layer = Dropout(0.5)(layer)
#     layer = Dense(1,name='out_layer')(layer)
#     layer = Activation('sigmoid')(layer)
#     model = Model(inputs=inputs,outputs=layer)
#     return model
# model = RNN()
# model.summary()
# model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy', recall, precision, fmeasure])

In [ ]:
train = pd.read_csv("/Users/isabelmetzger/PycharmProjects/n2c2/LSTM_BASELINES/data/train_May4th_binary.csv")
train_with_ID = train[['ID', 'text']]

y_train = train[["ABDOMINAL","ADVANCED_CAD","ALCOHOL_ABUSE",
               "ASP_FOR_MI","CREATININE","DIETSUPP_2MOS",
               "DRUG_ABUSE","ENGLISH","HBA1C","KETO_1YR",
               "MAJOR_DIABETES", "MAKES_DECISIONS", "MI_6MOS"]]
test = pd.read_csv("/Users/isabelmetzger/PycharmProjects/n2c2/LSTM_BASELINES/data/test_May4th_binary.csv")
test_with_ID = test[['ID', 'text']]

y_test = test[["ABDOMINAL","ADVANCED_CAD","ALCOHOL_ABUSE",
               "ASP_FOR_MI","CREATININE","DIETSUPP_2MOS",
               "DRUG_ABUSE","ENGLISH","HBA1C","KETO_1YR",
               "MAJOR_DIABETES", "MAKES_DECISIONS", "MI_6MOS"]]

# train = pd.read_csv("/Users/isabelmetzger/PycharmProjects/n2c2/train/train_May_141_samples.csv")

# test = pd.read_csv("/Users/isabelmetzger/PycharmProjects/n2c2/train/test_May_samples.csv")

train.head()

In [ ]:


print(test.head())

In [ ]:
import keras.backend as K

def binary_accuracy(y_true, y_pred):
    return K.mean(K.equal(y_true, K.round(y_pred)))


def categorical_accuracy(y_true, y_pred):
    return K.mean(K.equal(K.argmax(y_true, axis=-1),
                          K.argmax(y_pred, axis=-1)))


def sparse_categorical_accuracy(y_true, y_pred):
    return K.mean(K.equal(K.max(y_true, axis=-1),
                          K.cast(K.argmax(y_pred, axis=-1), K.floatx())))


def top_k_categorical_accuracy(y_true, y_pred, k=5):
    return K.mean(K.in_top_k(y_pred, K.argmax(y_true, axis=-1), k))


def mean_squared_error(y_true, y_pred):
    return K.mean(K.square(y_pred - y_true))


def mean_absolute_error(y_true, y_pred):
    return K.mean(K.abs(y_pred - y_true))


def mean_absolute_percentage_error(y_true, y_pred):
    diff = K.abs((y_true - y_pred) / K.clip(K.abs(y_true),
                                            K.epsilon(),
                                            None))
    return 100. * K.mean(diff)


def mean_squared_logarithmic_error(y_true, y_pred):
    first_log = K.log(K.clip(y_pred, K.epsilon(), None) + 1.)
    second_log = K.log(K.clip(y_true, K.epsilon(), None) + 1.)
    return K.mean(K.square(first_log - second_log))


def hinge(y_true, y_pred):
    return K.mean(K.maximum(1. - y_true * y_pred, 0.))


def squared_hinge(y_true, y_pred):
    return K.mean(K.square(K.maximum(1. - y_true * y_pred, 0.)))


def categorical_crossentropy(y_true, y_pred):
    return K.mean(K.categorical_crossentropy(y_pred, y_true))


def sparse_categorical_crossentropy(y_true, y_pred):
    return K.mean(K.sparse_categorical_crossentropy(y_pred, y_true))


def binary_crossentropy(y_true, y_pred):
    return K.mean(K.binary_crossentropy(y_pred, y_true))


def kullback_leibler_divergence(y_true, y_pred):
    y_true = K.clip(y_true, K.epsilon(), 1)
    y_pred = K.clip(y_pred, K.epsilon(), 1)
    return K.mean(K.sum(y_true * K.log(y_true / y_pred), axis=-1))


def poisson(y_true, y_pred):
    return K.mean(y_pred - y_true * K.log(y_pred + K.epsilon()))


def cosine_proximity(y_true, y_pred):
    y_true = K.l2_normalize(y_true, axis=-1)
    y_pred = K.l2_normalize(y_pred, axis=-1)
    return -K.mean(y_true * y_pred)


def matthews_correlation(y_true, y_pred):
    """Matthews correlation metric.
    It is only computed as a batch-wise average, not globally.
    Computes the Matthews correlation coefficient measure for quality
    of binary classification problems.
    """
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))
    y_pred_neg = 1 - y_pred_pos

    y_pos = K.round(K.clip(y_true, 0, 1))
    y_neg = 1 - y_pos

    tp = K.sum(y_pos * y_pred_pos)
    tn = K.sum(y_neg * y_pred_neg)

    fp = K.sum(y_neg * y_pred_pos)
    fn = K.sum(y_pos * y_pred_neg)

    numerator = (tp * tn - fp * fn)
    denominator = K.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))

    return numerator / (denominator + K.epsilon())


def precision(y_true, y_pred):
    """Precision metric.
    Only computes a batch-wise average of precision.
    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def recall(y_true, y_pred):
    """Recall metric.
    Only computes a batch-wise average of recall.
    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


def fbeta_score(y_true, y_pred, beta=1):
    """Computes the F score.
    The F score is the weighted harmonic mean of precision and recall.
    Here it is only computed as a batch-wise average, not globally.
    This is useful for multi-label classification, where input samples can be
    classified as sets of labels. By only using accuracy (precision) a model
    would achieve a perfect score by simply assigning every class to every
    input. In order to avoid this, a metric should penalize incorrect class
    assignments as well (recall). The F-beta score (ranged from 0.0 to 1.0)
    computes this, as a weighted mean of the proportion of correct class
    assignments vs. the proportion of incorrect class assignments.
    With beta = 1, this is equivalent to a F-measure. With beta < 1, assigning
    correct classes becomes more important, and with beta > 1 the metric is
    instead weighted towards penalizing incorrect class assignments.
    """
    if beta < 0:
        raise ValueError('The lowest choosable beta is zero (only precision).')

    # If there are no true positives, fix the F score at 0 like sklearn.
    if K.sum(K.round(K.clip(y_true, 0, 1))) == 0:
        return 0

    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    bb = beta ** 2
    fbeta_score = (1 + bb) * (p * r) / (bb * p + r + K.epsilon())
    return fbeta_score


def fmeasure(y_true, y_pred):
    """Computes the f-measure, the harmonic mean of precision and recall.
    Here it is only computed as a batch-wise average, not globally.
    """
    return fbeta_score(y_true, y_pred, beta=1)


# aliases
mse = MSE = mean_squared_error
mae = MAE = mean_absolute_error
mape = MAPE = mean_absolute_percentage_error
msle = MSLE = mean_squared_logarithmic_error
cosine = cosine_proximity
fscore = f1score = fmeasure
print("K backend imported!")

In [ ]:
list_sentences_train = train["text"].fillna("").values
list_classes = [""]
print(len(list_classes))

In [ ]:
#["ABDOMINAL", "ADVANCED_CAD", "CREATININE", "HBA1C",
#                 "MAJOR_DIABETES", "MI_6MOS"]
y = train[list_classes].values
max_features = 15500 #
maxlen = 7000 #padding length
num_folds = 2#2 #number of folds
list_sentences_test = test["text"].fillna("").values

print('mean text len:',train["text"].str.count('\S+').mean())
print('max text len:',train["text"].str.count('\S+').max())

#tokenizer = Tokenizer()
#tokenizer.fit_on_texts(list(list_sentences_train)) #  + list(list_sentences_test)
#num_words = sum([1 for _, v in tokenizer.word_counts.items() if v >= min_count])
#print('num_words',num_words)
#max_features = num_words
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train)) # + list(list_sentences_test)
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
print('padding sequences')
X_train = {}
X_test = {}
X_train['text'] = sequence.pad_sequences(list_tokenized_train, maxlen=maxlen, padding='post', truncating='post')
X_test['text'] = sequence.pad_sequences(list_tokenized_test, maxlen=maxlen, padding='post', truncating='post')

print('numerical variables')

#train['tx_numberer'] = train.text.apply(lambda s : sia.polarity_scores(s)['compound'])
#test['tx_numb'] = test.text.apply(lambda s : sia.polarity_scores(s)['compound'])
scaler = MinMaxScaler()

X_train['num_vars'] = scaler.fit_transform(train[['num_words','num_comas','num_bangs','num_quotas','avg_word', 'sentiment']])
X_test['num_vars'] = scaler.transform(test[['num_words','num_comas','num_bangs','num_quotas','avg_word', 'sentiment']])

all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()

print('create embedding matrix')
word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

In [ ]:
def get_model_cnn(X_train):
    global embed_size
    inp = Input(shape=(maxlen, ), name="text")
    num_vars = Input(shape=[X_train["num_vars"].shape[1]], name="num_vars")
    x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
    x = SpatialDropout1D(0.2)(x)
    z = GlobalMaxPool1D()(x)
    x = GlobalMaxPool1D()(Conv1D(embed_size, 4, activation="relu")(x))
    x = Concatenate()([x,z,num_vars])
    x = Dropout(0.3)(x)
    x = Dense(13, activation="sigmoid")(x)
    model = Model(inputs=[inp,num_vars], outputs=x)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', precision, recall, fmeasure])
    return model   

print("GLOVE KERAS SIMPLE MODEL! WITH FE! 300Dimension Glovee\n")
print('start modeling')
from sklearn import metrics
scores = []
predict = np.zeros((test.shape[0],13))
prediction_class = np.zeros((test.shape[0],13))
oof_predict = np.zeros((train.shape[0],13))
f1_scores = []
acc_scores = []
kf = KFold(n_splits=num_folds, shuffle=True, random_state=239)
for train_index, test_index in kf.split(X_train['num_vars']):
    kfold_X_train = {}
    kfold_X_valid = {}
    y_train,y_test = y[train_index], y[test_index]
    for c in ['text','num_vars']:
        kfold_X_train[c] = X_train[c][train_index]
        kfold_X_valid[c] = X_train[c][test_index]

    model = get_model_cnn(X_train)
    model.fit(kfold_X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1)
    predict += model.predict(X_test, batch_size=1000) / num_folds
    model.predict_generator()
    #predict_classes += predict.argmax(axis=-1)
    oof_predict[test_index] = model.predict(kfold_X_valid, batch_size=1000)
    
    #cv_score = metrics.roc_auc_score(y_test, oof_predict[test_index])
    #fscore =  metrics.f1_score(y_test, oof_predict[test_index])
    #metrics.average_precision_score(y_test, oof_predict[test_index])
    print(model.metrics_names)#, model.metrics_tensors)
    #scores.append(cv_score)
    #f1_scores.append(fscore)
    #print('cv score: ', cv_score)
    #print('f1 score: ',fscore)
    #print(classification_report(y_test, oof_predict[test_index]))

#print('Total CV score is {}'.format(np.mean(scores)))    
#print('Mean f1 score is {}'.format(np.mean(f1_scores))) 
sample_submission_classes = pd.DataFrame.from_dict({'ID': test['ID']})
sample_submission = pd.DataFrame.from_dict({'ID': test['ID']})
oof = pd.DataFrame.from_dict({'ID': train['ID']})
for c in list_classes:
    oof[c] = np.zeros(len(train))
    sample_submission[c] = np.zeros(len(test))
    #sample_submission_classes[c] = np.zeros(len(test))

#sample_submission_classes[list_classes] = prediction_class
#sample_submission.to_csv('11submit_cnn_avg_' + str(num_folds) + '_foldsMay13_predictions.csv', index=False)

In [ ]:
    
sample_submission[list_classes] = predict
    #class_prediction = target_to_change.argmax(axis=-1) #predict.argmax(axis=-1)
    #sample_submission_classes[class_name] = class_prediction

In [ ]:
sample_submission.head()

In [ ]:
for class_name in list_classes:
    pred_targ = sample_submission[class_name]
    pred_class = pred_targ
    sample_submission_classes[class_name] = pred_class
    
sample_submission_classes

In [ ]:
# prediction_classes = np.zeros((test.shape[0],13))

for ind, column in enumerate(sample_submission.):
    print(ind, column)
    
import sklearn
sklearn.utils.
#     sample_submission_classes[list_classes] = prediction_classes
#sample_submission_classes.to_csv('11CLASSESsubmit_cnn_avg_' + str(num_folds) + '_foldsMay13_predictions.csv', index=False)

In [ ]:
predicts_classes = predict.argmax(axis=-1)
sample_submission_classes[list_classes] = predicts_classes
sample_submission_classes.to_csv('11CLASSESsubmit_cnn_avg_' + str(num_folds) + '_foldsMay13_predictions.csv', index=False)


oof[list_classes] = oof_predict
# oof.to_csv('cnn_'+str(num_folds)+'_oof_May13.csv', index=False)
# print(oof.head())
# print("done!")

In [ ]:
# from sklearn.multiclass import LabelBinarizer
# from sklearn.cross_validation import train_test_split
    
#x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
#x_train.head()


In [ ]:
cols_target =["ABDOMINAL","ADVANCED_CAD","ALCOHOL_ABUSE",
               "ASP_FOR_MI","CREATININE","DIETSUPP_2MOS",
               "DRUG_ABUSE","ENGLISH","HBA1C","KETO_1YR",
               "MAJOR_DIABETES", "MAKES_DECISIONS", "MI_6MOS"]

In [ ]:
print('Total rows in test is {}'.format(len(test)))
print('Total rows in train is {}'.format(len(train)))
print(train[cols_target].sum())
print(test[cols_target].sum())

In [ ]:
# transform the test data using the earlier fitted vocabulary, into a document-term matrix
# test_X_dtm = vect.transform(test_X)
# # examine the document-term matrix from X_test
test_X_dtm

# Solving a multi-label classification problem
One way to approach a multi-label classification problem is to transform the problem into separate single-class classifier problems. This is known as 'problem transformation'. There are three methods:

Binary Relevance. This is probably the simplest which treats each label as a separate single classification problems. The key assumption here though, is that there are no correlation among the various labels.


Classifier Chains. In this method, the first classifier is trained on the input X. Then the subsequent classifiers are trained on the input X and all previous classifiers' predictions in the chain. This method attempts to draw the signals from the correlation among preceding target variables.


Label Powerset. This method transforms the problem into a multi-class problem where the multi-class labels are essentially all the unique label combinations. In our case here, where there are six labels, Label Powerset would in effect turn this into a 2^6 or 64-class problem. {Thanks Joshua for pointing out.}


# tfidf

In [ ]:
#from sklearn.model_selection import train_test_split


train_X_dtm = vect.transform(test_X)
# examine the document-term matrix from X_test
train_X_dtm
test_X_dtm = vect.transform(test_X)
# examine the document-term matrix from X_test
test_X_dtm
#X_train_tfidf, X_test_tfidf, y_train_tfidf, y_test_tfidf = train_test_split(X_dtm, train_df.text, train_size=0.75)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
sample_submission_NAIVEBAYES = pd.DataFrame.from_dict({'ID': test['ID']})
print('MULTINOMIAL NAIVE BAYES WITH TF-IDF MULTI LABEL CLASSIFICATION')
model_tfidf_NB = MultinomialNB()
accuracy_score_list = []
f1_score_list = []
val_f1_score_list = []
val_accuracy_score_list = []
for label in cols_target:
    print('\nPROCESSING {}'.format(label))
    y = train_df[label]
    test_y = test_df[label]
    model_tfidf_NB.fit(X_dtm, y)
    y_pred_X = model_tfidf_NB.predict(X_dtm)
    print('Training accuracy is {}'.format(metrics.accuracy_score(y, y_pred_X)))
    print('Training f1 binary is {}'.format(metrics.f1_score(y, y_pred_X, average='binary'))) 
    print('Training f1 macro is {}'.format(metrics.f1_score(y, y_pred_X, average='macro')))  
    print('Training f1 micro is {}'.format(metrics.f1_score(y, y_pred_X, average='micro')))  
    print('Training f1 weighted is {}'.format(metrics.f1_score(y, y_pred_X, average='weighted')))  
    #print('Training f1 avg=none is {}'.format(metrics.f1_score(y, y_pred_X, average=None)))
    test_y_prob = model_tfidf_NB.predict_proba(test_X_dtm)[:,1]
    test_y_pred = model_tfidf_NB.predict(test_X_dtm)
    #print(test_y_pred)
    accuracy_score_list.append(metrics.accuracy_score(y, y_pred_X))
    f1_score_list.append(metrics.f1_score(y, y_pred_X, average='binary'))
    #print('\nValidation accuracy is {}'.format(accuracy_score(test_y, test_y_pred)))
    
    print('\nVALIDATION CLASSIFICATION REPORT for {}'.format(label))
    print(metrics.classification_report(test_y, test_y_pred))
    val_accuracy_score_list.append(metrics.accuracy_score(test_y, test_y_pred))
    
print('MULTINOMIAL NAIVE BAYES WITH TF-IDF')
print('*************************************************************')
print('Mean Training Accuracy Score is {}'.format(np.mean(accuracy_score_list)))
print('Mean Training F1 Score is {}'.format(np.mean(f1_score_list)))
print('*************************************************************')
print('Mean Validation Accuracy Score is {}'.format(np.mean(val_accuracy_score_list)))

Now on to next model


In [ ]:
cols_target

In [ ]:
import pandas as pd
submission_binary_MultinomialNaiveBayes = pd.DataFrame(columns=cols_target)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
logreg = LogisticRegression(C=12.0)

print('LOGISTIC REGRESSION MODEL')
f1_scores_list = []
accuracy_list = []
val_accuracy_score_list = []
for label in cols_target:
    print('... Processing {}'.format(label))
    y = train_df[label]
    test_y = test_df[label]
    # train the model using X_dtm & y
    logreg.fit(X_dtm, y)
    # compute the training accuracy
    y_pred_X = logreg.predict(X_dtm)
    print('Training accuracy is {}'.format(accuracy_score(y, y_pred_X)))
    print('Training f1 score is is {}'.format(f1_score(y, y_pred_X, pos_label=1)))
    #print('Training f1 macro score is is {}'.format(f1_score(y, y_pred_X), 'macro'))
    #print('Training f1 micro score is is {}'.format(f1_score(y, y_pred_X), 'micro'))
    #print('Training f1 weighted score is is {}'.format(f1_score(y, y_pred_X), 'weighted'))
    # compute the predicted probabilities for X_test_dtm
    test_y_prob = logreg.predict_proba(test_X_dtm)[:,1]
    test_y_pred = logreg.predict(test_X_dtm)
    submission_binary[label] = test_y_prob
    #print(test_y_pred)
    accuracy_list.append(metrics.accuracy_score(y, y_pred_X))
    f1_scores_list.append(metrics.f1_score(y, y_pred_X, pos_label=1))
    
    print('\nVALIDATION CLASSIFICATION REPORT for {}'.format(label), '\n')
    print(metrics.classification_report(test_y, test_y_pred))
    val_accuracy_score_list.append(metrics.accuracy_score(test_y, test_y_pred))
    

print('*************************************************************')
print('Mean Training Accuracy Score is {}'.format(np.mean(accuracy_list)))
print('Mean Training F1 Score is {}'.format(np.mean(f1_scores_list)))
print('*************************************************************')
print('\nMean Validation Accuracy Score is {}'.format(np.mean(val_accuracy_score_list)))
    

In [ ]:
submission_binary.head()

In [ ]:
submission_binary.to_csv("submission_binary.csv")

In [ ]:
# create submission file
submission_chains = pd.read_csv('submission_binary.csv')

# create a function to add features
def add_feature(X, feature_to_add):
    '''
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    '''
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

In [ ]:

for label in cols_target:
    print('... Processing {}'.format(label))
    y = train_df[label]
    # train the model using X_dtm & y
    logreg.fit(X_dtm,y)
    # compute the training accuracy
    y_pred_X = logreg.predict(X_dtm)
    print('Training Accuracy is {}'.format(accuracy_score(y,y_pred_X)))
    print('Training f1 score is {}'.format(f1_score(y,y_pred_X)))
    # make predictions from test_X
    test_y = logreg.predict(test_X_dtm)
    test_y_prob = logreg.predict_proba(test_X_dtm)[:,1]
    submission_chains[label] = test_y_prob
    # chain current label to X_dtm
    X_dtm = add_feature(X_dtm, y)
    print('Shape of X_dtm is now {}'.format(X_dtm.shape))
    # chain current label predictions to test_X_dtm
    test_X_dtm = add_feature(test_X_dtm, test_y)
    print('Shape of test_X_dtm is now {}'.format(test_X_dtm.shape))

In [ ]:
submission_chains.head()

In [ ]:
# generate submission file
submission_chains.to_csv('submission_chains.csv', index=False)

In [ ]:
submission_combined = pd.DataFrame(columns=cols_target)
for label in cols_target:
    submission_combined[label] = 0.5*(submission_chains[label]+submission_binary[label])

submission_combined.head()

In [ ]:
submission_combined.to_csv('submission_combined.csv', index=False)

Building the model
We'll start by creating a bag of words representation, as a term document matrix. We'll use ngrams, as suggested in the NBSVM paper.

In [ ]:
text = 'text'
train_df = train
test_df = test
train_df[text].fillna("unknown", inplace=True)
test_df[text].fillna("unknown", inplace=True)

import re, string
re_tok = re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])')
def tokenize(s):
    return re_tok.sub(r' \1 ', s).split()

import pandas as pd, numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import re, string
n = train_df.shape[0]
vec = TfidfVectorizer(ngram_range=(1,2), tokenizer=tokenize,
               min_df=3, max_df=0.9, strip_accents='unicode', use_idf=1,
               smooth_idf=1, sublinear_tf=1 )
trn_term_doc = vec.fit_transform(train_df[text])
test_term_doc = vec.transform(test_df[text])
re_tok = re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])')
def tokenize(s):
    return re_tok.sub(r' \1 ', s).split()
n = train_df.shape[0]

vec = TfidfVectorizer(ngram_range=(1,3), tokenizer=tokenize,
               min_df=3, max_df=0.9, strip_accents='unicode', use_idf=1,
               smooth_idf=1, sublinear_tf=1 )
trn_term_doc = vec.fit_transform(train_df[text])
test_term_doc = vec.transform(test_df[text])
trn_term_doc, test_term_doc

In [ ]:
cols_target = ['ABDOMINAL',
 'ADVANCED_CAD',
 'ALCOHOL_ABUSE',
 'ASP_FOR_MI',
 'CREATININE',
 'DIETSUPP_2MOS',
 'DRUG_ABUSE',
 'ENGLISH',
 'HBA1C',
 'MAJOR_DIABETES',
 'MAKES_DECISIONS',
 'MI_6MOS']
from sklearn import metrics
def pr(y_i, y):
    p = x[y==y_i].sum(0)
    return (p+1) / ((y==y_i).sum()+1)

x = trn_term_doc
test_x = test_term_doc
def get_mdl(y):
    y = y.values
    r = np.log(pr(1,y) / pr(0,y))
    m = LogisticRegression(C=4, dual=True)
    x_nb = x.multiply(r)
    return m.fit(x_nb, y), r

preds = np.zeros((len(test_df), len(cols_target)))

acc_list = []
f1_list = []
for i, j in enumerate(cols_target):
    print('... Processing {}'.format(j))
    print('fit', j)
    m,r = get_mdl(train_df[j])
    preds[:,i] = m.predict_proba(test_x.multiply(r))[:,1]
    prediction_y = m.predict(test_x.multiply(r))
    y_test = test_df[j]
    print(metrics.classification_report(y_test, prediction_y))
    f1 = metrics.f1_score(y_test, prediction_y, pos_label=1)
    f1_list.append(f1)
    print(f1)
    


print('Total f1 score is {}'.format(np.mean(f1_list)))
print('Total acc score is {}'.format(np.mean(acc_list)))
    
    

In [ ]:

predictions = pd.concat([pd.DataFrame(preds, columns = cols_target)], axis=1)

predictions.head()

In [ ]:
Tfidf_sklearn_model_predictions = predictions

Tfidf_sklearn_model_predictions.to_csv("Tfidf_sklearn_model_predictions.csv")

In [ ]:
# using binary relevance
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.naive_bayes import GaussianNB

# initialize binary relevance multi-label classifier
# with a gaussian naive bayes base classifier
#classifier = BinaryRelevance(GaussianNB())

# train
# classifier.fit(trn_term_doc, test_term_doc)

# # predict
# predictions = classifier.predict(trn_term_doc)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score


In [ ]:
# using binary relevance
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.naive_bayes import GaussianNB

# initialize binary relevance multi-label classifier
# with a gaussian naive bayes base classifier
classifier = BinaryRelevance(GaussianNB())
for label in cols_target:
    print('Processing {}'.format(label))
    y = train_df[label]
    classifier.fit(X_train_tfidf, y)
    y_pred_X = classifier.predict(X_dtm)
    print('Training accuracy is {}'.format(accuracy_score(y, y_pred_X)))
    print('Training f1 macro is {}'.format(f1_score(y, y_pred_X, average='macro')))  
    print('Training f1 micro is {}'.format(f1_score(y, y_pred_X, average='micro')))  
    print('Training f1 weighted is {}'.format(f1_score(y, y_pred_X, average='weighted')))  
    test_y_prob = classifier.predict_proba(test_X_dtm)[:,1]


# train
#classifier.fit(X_train_tfidf, y_train_tfidf)

# predict
predictions = classifier.predict(X_test_tfidf)


metrics.accuracy_score(y_test,predictions)
from sklearn.metrics import f1_score
metrics.f1_score(y_test_tfidf, predictions)